<a href="https://colab.research.google.com/github/Hanmant9561/Exploratory-Data-Analysis-Retail-Level-Beginner/blob/main/Black_Coffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Loading the Libraries
import pandas as pd
import requests
import string
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('punkt')
nltk.download("stopwords")
import textstat
from textblob import TextBlob
import spacy
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# Loading the EXCEL file.
url_data = pd.read_csv(r'/content/Input.xlsx - Sheet1.csv')
print(url_data)

     URL_ID                                                URL
0        37  https://insights.blackcoffer.com/ai-in-healthc...
1        38  https://insights.blackcoffer.com/what-if-the-c...
2        39  https://insights.blackcoffer.com/what-jobs-wil...
3        40  https://insights.blackcoffer.com/will-machine-...
4        41  https://insights.blackcoffer.com/will-ai-repla...
..      ...                                                ...
109     146  https://insights.blackcoffer.com/blockchain-fo...
110     147  https://insights.blackcoffer.com/the-future-of...
111     148  https://insights.blackcoffer.com/big-data-anal...
112     149  https://insights.blackcoffer.com/business-anal...
113     150  https://insights.blackcoffer.com/challenges-an...

[114 rows x 2 columns]


In [28]:
# Appending links into the list.
links = [ i for i in url_data['URL']]

In [19]:
# Scraping the content from the links.
reports = []
for url in links:
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',}
    r = requests.get(url, headers=headers)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    reports.append(soup.get_text())
print(f'Total {len(reports)} reports have been saved')


ps = PorterStemmer()

Total 114 reports have been saved


In [20]:
# Textual EDA.
corpus = []
for i in reports:
    review = re.sub('[^a-zA-z]', ' ', i)
    review = review.lower()
    review = review.split()
    review = [word for word in review if word not in string.punctuation]
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

analyzer = SentimentIntensityAnalyzer()

In [21]:
# Compound/Neg/Pos/Neu Score Calculation.
em = []
for i in range(len(corpus)): 
  vs = analyzer.polarity_scores(corpus[i])
  em.append(vs)

In [30]:
# Appending the result into the senti_data and setting URL as index.  
senti_data = pd.DataFrame(em)

senti_data['URL'] = url_data['URL']
senti_data = senti_data.set_index('URL')

senti_data.head()

,neg,neu,pos,compound
URL,,,,
https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/,0.030,0.881,0.089,0.9998
https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/,0.031,0.878,0.091,0.9997
https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/,0.033,0.880,0.087,0.9997
https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/,0.027,0.886,0.087,0.9998
https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/,0.041,0.867,0.093,0.9997


In [31]:
# Calculation of Polarity and Subjectivity Score.
polarity = []
subjectivity = []
for i in range(len(corpus)): 
  p = TextBlob(corpus[i]).sentiment[0]
  s = TextBlob(corpus[i]).sentiment[1]
  polarity.append(p)
  subjectivity.append(s)
senti_data['polarity'] = polarity
senti_data['subjectivity'] = subjectivity

In [32]:
# Calculating the Average length of Sentence.
def avg_sentence_len(texts):
  a = []
  for text in texts :
    sentences = text.split(".") 
    words = text.split(" ") 
    if(sentences[len(sentences)-1]==""): 
      average_sentence_length = len(words) / len(sentences)-1
    else:
      average_sentence_length = len(words) / len(sentences)
    a.append(average_sentence_length)
  return a
avg_sentence = avg_sentence_len(corpus)
senti_data['average sentence length'] = avg_sentence

In [33]:
# Calculating the Percentage of complex words in the corpus.
complexity = []
for i in range(len(corpus)):
  va = textstat.spache_readability(corpus[i])*(10**-1)
  complexity.append(va)
senti_data['percentage of complex words'] = complexity

In [34]:
# Calculating the Word count
word = []
for i in range(len(corpus)):
  va = textstat.lexicon_count(corpus[i], removepunct=True)
  word.append(va)
senti_data['word count'] = word

In [35]:
# Calculating the Fog Index
fog = []
for i in range(len(corpus)):
  va = textstat.gunning_fog(corpus[i])
  fog.append(va)
senti_data['fog index'] = fog

In [36]:
# Calculating the Syllables
syllable = []
for i in range(len(corpus)):
  va = textstat.monosyllabcount(corpus[i])
  syllable.append(va)
senti_data['syllable per word'] = syllable

In [37]:
# Calculating the Average Word Length.
def avg_word(text):
  count = []
  for i in range(len(text)):
    words = text[i].split()
    count.append(round(sum(len(word) for word in words)/len(words), 3))
  return count
senti_data['average word length'] = avg_word(corpus)

In [38]:
# Calculating the Number of Personal Pronouns.
nlp = spacy.load("en_core_web_sm")
c = []
for i in range(len(corpus)): 
  doc = nlp(corpus[i])
  a = [] 
  for token in doc:
    if token.pos_ =='PRON' : #, token.tag_, tok)
        b = []
        a.append(token)  
  c.append(len(a))
senti_data['personal pronouns'] = c

In [39]:
# Exporting the final results into EXCEL file.
senti_data.to_excel('Output Data Structure.xlsx')

In [40]:
data=pd.read_excel(r"/content/Output Data Structure.xlsx")

In [41]:
data

,URL,neg,neu,pos,compound,polarity,subjectivity,average sentence length,percentage of complex words,word count,fog index,syllable per word,average word length,personal pronouns
0,https://insights.blackcoffer.com/ai-in-healthc...,0.030,0.881,0.089,0.9998,0.056823,0.404649,5820,82.235,5818,2329.43,2704,6.411,65
1,https://insights.blackcoffer.com/what-if-the-c...,0.031,0.878,0.091,0.9997,0.074609,0.376238,5330,75.314,5328,2133.38,2602,6.403,92
2,https://insights.blackcoffer.com/what-jobs-wil...,0.033,0.880,0.087,0.9997,0.053590,0.378979,5577,78.806,5575,2232.35,2596,6.506,87
3,https://insights.blackcoffer.com/will-machine-...,0.027,0.886,0.087,0.9998,0.059979,0.372876,5503,77.751,5501,2202.55,2660,6.380,129
4,https://insights.blackcoffer.com/will-ai-repla...,0.041,0.867,0.093,0.9997,0.059844,0.382461,5907,83.449,5905,2364.12,2928,6.233,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,https://insights.blackcoffer.com/blockchain-fo...,0.029,0.886,0.085,0.9997,0.048056,0.383053,4954,70.022,4952,1983.13,2305,6.697,108
110,https://insights.blackcoffer.com/the-future-of...,0.024,0.890,0.086,0.9998,0.052904,0.403412,5404,76.361,5402,2163.04,2509,6.505,122
111,https://insights.blackcoffer.com/big-data-anal...,0.041,0.874,0.086,0.9994,0.041351,0.387964,5194,73.398,5192,2079.11,2380,6.589,110
112,https://insights.blackcoffer.com/business-anal...,0.022,0.890,0.088,0.9998,0.073034,0.394948,4962,70.127,4960,1986.30,2263,6.656,85
